# 选择参数并将 domain1 的源图像迁移到至少 3 个不同的域。

* $\alpha$ ：  二维矩阵掩码控制着在幅度频谱图中有多少尺度的低频信息会被交换，在中心比例为 $\alpha$ 处为1，其他地方为0
* $\lambda$：该参数控制着调整分布信息的内插比例

In [128]:
import os
from sklearn.manifold import TSNE
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import random

alpha =0.2
lam = 0.5
target_domain = r'Domain4'    
source_domain_folder = r'G:\FedICRA\data\FAZ\Domain1\train\imgs'

In [129]:
## 得到所有源域内的图像路径，在该函数中是domain1
def get_source_domain_paths(folder):
    source_domain_paths = []
    for root, dirs, files in os.walk(folder):
        for file in files:
            file_path = os.path.join(root, file)
            # 判断文件是否为图像文件（这里以常见的图片格式为例，可以根据实际情况扩展）
            if file_path.lower().endswith(('.png')):
                source_domain_paths.append(file_path)
    return source_domain_paths

source_domain_paths = get_source_domain_paths(folder)

In [130]:
# 从目标域中随机获得一个频谱图，在该例子中是domain2，project要求的是3个域
def get_random_target_domain_image_amp(target_domain):
    target_domain_paths = []
    target_domain = r'Domain2'
    current_directory = os.getcwd()  # 获取当前工作目录

    save_directory = os.path.join(current_directory, target_domain)  # 假设保存在Domain文件夹
    for root, dirs, files in os.walk(save_directory):
        for file in files:
            file_path = os.path.join(root, file)
            # 判断文件是否为图像文件
            if 'amp' in file_path.lower():
                target_domain_paths.append(file_path)


    random_target_domain_path = random.choice(target_domain_paths)
    img = cv2.imread(random_target_domain_path)
    # img_resize = cv2.resize(img, (200,200))
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)    # 灰度调整
    spect = np.fft.fft2(gray_image)
    spect = np.fft.fftshift(spect)  # 频谱中心化
    spect_amp_target = np.abs(spect)

    return spect_amp_target 



In [131]:
def get_source_domain_image_spec(image_path):
    img = cv2.imread(image_path)
    # img_resize = cv2.resize(img, (200,200))
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)    # 灰度调整
    spect = np.fft.fft2(gray_image)
    spect = np.fft.fftshift(spect)  # 频谱中心化
    spect_amp_source = np.abs(spect)
    spect_pha_source = np.angle(spect)

    return spect_amp_source,spect_pha_source

# spect_amp_source,spect_pha_source = get_source_domain_image_spec(r'G:\FedICRA\data\FAZ\Domain1\train\imgs\002_D_10.png')

In [132]:

# 生成掩码矩阵M，参数由shape和alpha进行调整，一个控制矩阵的大小，一个控制中心掩码区（全部为1）的比例 
def generate_binary_mask(shape, alpha):
    alpha  /= 2
    h, w = shape
    center_h, center_w = h//2, w//2
    mask = np.zeros((h,w))

    alpha_h = int(alpha * h) 
    alpha_w = int(alpha * w) 

    mask[center_h - alpha_h:center_h+alpha_h, center_w - alpha_w:center_w + alpha_w] = 1 

    return mask


In [133]:
# 由文章公式给出计算生成频谱图的代码
def get_spect_amp_generated(alpha: float, lam: float, spect_amp_source, spect_amp_target):
    mask = generate_binary_mask(spect_amp_source.shape,alpha)
    print(spect_amp_source.shape)
    spect_amp_generated = mask * ((1-lam)*spect_amp_source+lam*spect_amp_target) + (1-mask) * spect_amp_source
    return spect_amp_generated




In [134]:
def create_directory(target_domain):
    current_directory = os.getcwd()  # 获取当前工作目录

    save_directory = os.path.join(current_directory, f'Domain1_to_{target_domain}')  # 假设保存在Domain文件夹下
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    return save_directory 


In [135]:

def domain_generalization(source_domain_folder: str,target_domain: str, alpha:float, lam:float):
    '''
    领域泛化的包装函数，用于实现对某个源图像域的领域泛化。

    参数：
    source_domain_folder: str
        源域图像的文件夹路径
    target_domain: str
        表示目标域的字符串，'Domain2','Domain3','Domain4', 'Domain5' 
    返回值：无返回值
    '''
    # 生成新的文件夹
    generated_image_folder = create_directory(target_domain)
    source_domain_paths = get_source_domain_paths(source_domain_folder)
    # 打印所有图像文件的绝对路径
    for image_path in source_domain_paths:
        spect_amp_source, spect_pha_source = get_source_domain_image_spec(image_path)
        spect_amp_target = get_random_target_domain_image_amp(target_domain)
        spect_amp_generated = get_spect_amp_generated(alpha, lam, spect_amp_source,spect_amp_target)
        
        complex_spectrum = spect_amp_generated * np.exp(1j * spect_pha_source)
        # 逆频谱中心化
        reconstructed_image = np.fft.ifftshift(complex_spectrum)

        # 应用逆傅里叶变换
        reconstructed_image = np.fft.ifft2(reconstructed_image)
        # 生成图像
        reconstructed_image = np.abs(reconstructed_image).astype(np.uint8)
        # save
        parts = image_path.split('\\')  # 使用 '\\' 进行路径分割，得到每个部分

        img_name = parts[-1]
        generated_image_path = os.path.join(generated_image_folder,img_name) # 假设保存在Domain文件夹下\
        cv2.imwrite(generated_image_path,reconstructed_image)

domain_generalization(source_domain_folder=source_domain_folder,target_domain=target_domain,alpha=alpha,lam=lam)

(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)